In [1]:
from deepface import DeepFace

In [54]:
result = DeepFace.find(img_path = "./tests/im.jpg", db_path = "./db_rostos")

Finding representations: 100%|██████████| 2/2 [00:00<00:00,  4.57it/s]


Representations stored in ./db_rostos/representations_vgg_face.pkl file.Please delete this file when you add new identities in your database.
find function lasts  0.5922670364379883  seconds


É necessário excluir o arquivo representations_vgg_face.pkl quando adicionar uma pessoa na base de dados!

In [55]:
result[0].identity[0].split('/')[2].split('.')[0]

'Pedro'

In [58]:
DeepFace.analyze('db_rostos\Eduardo.jpg')

facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: C:\Users\Eduardo\.deepface\weights\facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 8.25MB/s]


age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: C:\Users\Eduardo\.deepface\weights\age_model_weights.h5
100%|██████████| 539M/539M [01:05<00:00, 8.23MB/s] 


gender_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: C:\Users\Eduardo\.deepface\weights\gender_model_weights.h5
100%|██████████| 537M/537M [00:59<00:00, 8.99MB/s] 


race_model_single_batch.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/race_model_single_batch.h5
To: C:\Users\Eduardo\.deepface\weights\race_model_single_batch.h5
100%|██████████| 537M/537M [01:01<00:00, 8.77MB/s] 
Action: race: 100%|██████████| 4/4 [00:00<00:00,  4.96it/s]   


[{'emotion': {'angry': 0.0033791344321798533,
   'disgust': 9.746427664936164e-07,
   'fear': 0.05282884812913835,
   'happy': 99.9005138874054,
   'sad': 0.030833299388177693,
   'surprise': 0.00038705125007254537,
   'neutral': 0.012057174171786755},
  'dominant_emotion': 'happy',
  'region': {'x': 494, 'y': 75, 'w': 272, 'h': 272},
  'age': 29,
  'gender': {'Woman': 1.3806403614580631, 'Man': 98.6193597316742},
  'dominant_gender': 'Man',
  'race': {'asian': 3.6306252582107845e-05,
   'indian': 0.00019986882762168534,
   'black': 2.2265163934775956e-06,
   'white': 99.18457865715027,
   'middle eastern': 0.4965700674802065,
   'latino hispanic': 0.3186155343428254},
  'dominant_race': 'white'}]